# Import Libraries

In [1]:
# Import Libraries
import os
import re
import time
import json
import nltk
import torch
import random
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)
import math
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import DataParallel
import torch.optim as optim
from tqdm import tqdm
from transformers import GPT2Model, GPT2Config
import seaborn as sns
import matplotlib.pyplot as plt
from rouge import Rouge



# Model Configuration

In [2]:
class Args():
    def __init__(self):
        # load train dataset
        self.dataset_dir = r"D:\Yeshiva\Spring24\AI\katzbot"
        
        self.train_path = r"dataset_katzbot/New_Train_QA_Pairs.csv"
        self.input_file_path = os.path.join(self.dataset_dir, self.train_path)
        
        # load test dataset
        self.test_path = r"dataset_katzbot/Test_QA_Pairs.csv"
        self.test_data = os.path.join(self.dataset_dir, self.test_path)
        self.save_text_file = "predicted_answer.txt"
        self.save_text_file_rlhf = "predicted_answer_rlhf.txt"
        self.predict_file = "Test_QA_Pairs_pretrained_gpt.csv"
        self.eval_results = "eval_results.csv"
        self.clf_data = "clf_dataset.csv"
        
        self.original_text = "answer"
        self.predicted_result = "predicted_answer"
        self.predicted_result_2 = "predicted_answer_2"

        
        # list of gpt model 
        self.gpt_model = ["gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl"]
        self.gpt = self.gpt_model[0]
        # # Model and tokeniser
        self.model_type = self.gpt
        self.tokenizer_name = self.gpt
        
        # Sets the directory to which the trained model will be saved
        self.output_dir = 'models_pretrained_4'
        self.model_name = "pretrained_gpt2.pt"

        # rlhf model
        self.model_rlhf_name = 'bert-base-uncased'
        
        
        # Parameters
        # Set the seed value all over the place to make this reproducible.
        self.seed_val = 42
        self.batch_size = 4
        self.epochs = 60
        self.learning_rate = 5e-4
        self.warmup_steps = 1e2
        self.epsilon = 1e-5
        self.weight_decay = 1e-2
        self.sample_every = 100
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


        # bert ppo
        self.N_GPUS = 1
        self.TRAIN_EPOCHS = 2
        self.TRAIN_BATCH_SIZE = 8
        self.EVAL_BATCH_SIZE = 8
        self.MAX_LEN = 128
        self.LOGGING_STEPS = 64
        self.SAVE_STEPS = 10240  # reduce it to a smaler value like 512 if you want to save checkpoints
        self.SAVE_TOTAL_LIMIT = 2

        self.set_seed = 123
        
args = Args()


In [41]:
from transformers import BertForSequenceClassification
from transformers import TrainingArguments
from transformers import BertTokenizerFast
from datasets import load_dataset
from transformers import pipeline
from transformers import set_seed
from transformers import Trainer
from datasets import DatasetDict
import transformers
import numpy as np
import datasets
import logging 
import torch
import os

In [42]:
# creates an instance of the BertForSequenceClassification class from the Hugging Face Transformers library.
model_bert = BertForSequenceClassification.from_pretrained(args.model_rlhf_name, num_labels=2,  force_download=True)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sahil\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
tokenizer = BertTokenizerFast.from_pretrained(args.model_rlhf_name)

In [44]:
test_clf

'models_pretrained_4\\clf_dataset.csv'

In [45]:
from datasets import load_dataset
from datasets import DatasetDict
from transformers import BertTokenizerFast
import os
import functools

# # Define your tokenizer
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# # Define your preprocess function
# def preprocess_function(examples, tokenizer):
#     return tokenizer(examples['response'], truncation=True, padding=True)

def preprocess_function(examples, tokenizer):
    # Extract response text from examples
    responses = examples['response']
    
    # Encode the response text using tokenizer.batch_encode_plus
    encodings = tokenizer.batch_encode_plus(
        responses,  # List of response texts
        truncation=True,
        padding=True
    )
    
    # Return the encoded inputs
    return encodings

# Load your dataset
data = load_dataset('csv', 
                    data_files=test_clf, 
                    delimiter=',', 
                    split='train', 
                    cache_dir='/tmp/cache')

# Split your dataset
train_validation_test = data.train_test_split(shuffle=True, seed=123, test_size=0.1)
data_splits = DatasetDict({'train': train_validation_test['train'],  
                           'validation': train_validation_test['test']})

# Partially apply the tokenizer to the preprocess function
partial_preprocess_function = functools.partial(preprocess_function, tokenizer=tokenizer)

# Tokenize datasets 
num_proc = int(os.cpu_count()/args.N_GPUS)
tokenized_data = data_splits.map(partial_preprocess_function, 
                                 batched=True, 
                                 num_proc=num_proc)

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=32):   0%|          | 0/2703 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/301 [00:00<?, ? examples/s]

# Finetune
Define training hyperparameters

In [46]:
# Set various training parameters such as the number of training epochs, batch size, and logging options.
training_args = TrainingArguments(output_dir=args.output_dir + '/train', 
                                  overwrite_output_dir=True, 
                                  num_train_epochs=args.epochs,  
                                  optim='adamw_torch', 
                                  save_strategy='steps', 
                                  evaluation_strategy='epoch',
                                  per_device_train_batch_size=args.TRAIN_BATCH_SIZE, 
                                  per_device_eval_batch_size=args.EVAL_BATCH_SIZE, 
                                  warmup_steps=args.warmup_steps, 
                                  weight_decay=args.weight_decay,
                                  logging_steps=args.LOGGING_STEPS,
                                  save_steps=args.SAVE_STEPS, 
                                  save_total_limit=args.SAVE_TOTAL_LIMIT,
                                  logging_dir='logs')

### Training
P.S: Since our dataset is small it has overfits

In [47]:
import time

trainer = Trainer(model=model_bert, 
                  args=training_args, 
                  train_dataset=tokenized_data['train'], 
                  eval_dataset=tokenized_data['validation'], 
                  tokenizer=tokenizer)

start_time = time.time()
trainer.train()
end_time = time.time()

total_training_time = end_time - start_time

# Convert total_training_time to hours, minutes, and seconds
hours = int(total_training_time // 3600)
minutes = int((total_training_time % 3600) // 60)
seconds = int(total_training_time % 60)

print(f"Total training time: {hours}:{minutes}:{seconds}")


c:\Users\sahil\.conda\envs\genai\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sahilkumar158 (sahilthegnius). Use `wandb login --relogin` to force relogin


  0%|          | 0/20280 [00:00<?, ?it/s]

{'loss': 0.713, 'grad_norm': 9.705682754516602, 'learning_rate': 3.2000000000000005e-05, 'epoch': 0.19}
{'loss': 0.6098, 'grad_norm': 4.184609413146973, 'learning_rate': 4.993062438057483e-05, 'epoch': 0.38}
{'loss': 0.643, 'grad_norm': 7.928568363189697, 'learning_rate': 4.977205153617443e-05, 'epoch': 0.57}
{'loss': 0.6194, 'grad_norm': 8.016056060791016, 'learning_rate': 4.961347869177403e-05, 'epoch': 0.76}
{'loss': 0.6349, 'grad_norm': 7.96769380569458, 'learning_rate': 4.945490584737364e-05, 'epoch': 0.95}


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.585211455821991, 'eval_runtime': 0.4738, 'eval_samples_per_second': 635.263, 'eval_steps_per_second': 80.199, 'epoch': 1.0}
{'loss': 0.6263, 'grad_norm': 5.150811195373535, 'learning_rate': 4.929633300297324e-05, 'epoch': 1.14}
{'loss': 0.6139, 'grad_norm': 3.554089069366455, 'learning_rate': 4.913776015857285e-05, 'epoch': 1.33}
{'loss': 0.5955, 'grad_norm': 8.724480628967285, 'learning_rate': 4.8979187314172446e-05, 'epoch': 1.51}
{'loss': 0.6038, 'grad_norm': 5.327426910400391, 'learning_rate': 4.882061446977205e-05, 'epoch': 1.7}
{'loss': 0.6136, 'grad_norm': 4.9745306968688965, 'learning_rate': 4.866204162537166e-05, 'epoch': 1.89}


...

{'eval_loss': 4.568084239959717, 'eval_runtime': 0.4525, 'eval_samples_per_second': 665.201, 'eval_steps_per_second': 83.979, 'epoch': 58.0}
{'loss': 0.0037, 'grad_norm': 0.00019471711129881442, 'learning_rate': 1.565906838453915e-06, 'epoch': 58.13}
{'loss': 0.0, 'grad_norm': 0.00019216921646147966, 'learning_rate': 1.4073339940535184e-06, 'epoch': 58.32}
{'loss': 0.0037, 'grad_norm': 0.00025626926799304783, 'learning_rate': 1.2487611496531221e-06, 'epoch': 58.51}
{'loss': 0.0034, 'grad_norm': 0.00016013451386243105, 'learning_rate': 1.0901883052527254e-06, 'epoch': 58.7}
{'loss': 0.0036, 'grad_norm': 0.00017847222625277936, 'learning_rate': 9.316154608523291e-07, 'epoch': 58.89}


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 4.578721046447754, 'eval_runtime': 0.4484, 'eval_samples_per_second': 671.245, 'eval_steps_per_second': 84.742, 'epoch': 59.0}
{'loss': 0.0, 'grad_norm': 0.00019013193377759308, 'learning_rate': 7.730426164519327e-07, 'epoch': 59.08}
{'loss': 0.0007, 'grad_norm': 0.00018648729019332677, 'learning_rate': 6.144697720515362e-07, 'epoch': 59.27}
{'loss': 0.0, 'grad_norm': 0.00014438832295127213, 'learning_rate': 4.5589692765113977e-07, 'epoch': 59.46}
{'loss': 0.0007, 'grad_norm': 0.0001576225331518799, 'learning_rate': 2.973240832507433e-07, 'epoch': 59.64}
{'loss': 0.0061, 'grad_norm': 0.00017922157712746412, 'learning_rate': 1.387512388503469e-07, 'epoch': 59.83}


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 4.582746505737305, 'eval_runtime': 0.4314, 'eval_samples_per_second': 697.799, 'eval_steps_per_second': 88.094, 'epoch': 60.0}
{'train_runtime': 1389.1812, 'train_samples_per_second': 116.745, 'train_steps_per_second': 14.599, 'train_loss': 0.11268299733740154, 'epoch': 60.0}
Total training time: 0:23:9


# PPO
Defining a configuration object for the Proximal Policy Optimization (PPO) algorithm

In [48]:
from trl import AutoModelForCausalLMWithValueHead
from transformers import GPT2Tokenizer
from transformers import set_seed
from datasets import load_dataset
from transformers import pipeline
from datasets import Dataset
from random import choices
from trl import PPOTrainer
from trl import PPOConfig
from evaluate import load
from tqdm import tqdm
import transformers 
import pandas as pd
import numpy as np
import bert_score
import ipywidgets
import datasets
import evaluate
import logging
import jupyter
import random
import torch
import wandb
import time
import trl
import os

In [49]:
pd.options.display.max_colwidth = None
np.random.seed(123)
tqdm.pandas()
set_seed = args.set_seed

In [50]:
config = PPOConfig(model_name=args.output_dir + '/train/finetuned', 
                   batch_size=args.batch_size,
                   learning_rate=args.learning_rate, #1.41e-6,
                   forward_batch_size=args.batch_size, # This parameter is useful when using large models that don't fit into memory.
                   remove_unused_columns=False, # All columns in the input data will be used during training.
                   log_with='wandb'# For logging training metrics
                  )

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\trl\trainer\ppo_config.py:150: UserWarning: Note that using `forward_batch_size` is deprecated, use `mini_batch_size` instead. By setting it you overwrite `mini_batch_size` which affects both the batch size during forward passes and also the mini batch size for PPO optimization.
  warnings.warn(


In [51]:

active_model = AutoModelForCausalLMWithValueHead.from_pretrained(args.output_dir)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(args.output_dir)

In [52]:
args.test_data

'D:\\Yeshiva\\Spring24\\AI\\katzbot\\dataset_katzbot/Test_QA_Pairs.csv'

In [53]:
dataset = load_dataset('csv', 
                       data_files=args.test_data,  
                       delimiter=',', 
                       split='train[:100%]',
                       download_mode='force_redownload')
dataset

Generating train split:   0%|          | 0/2081 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer'],
    num_rows: 2081
})

In [54]:
def tokenize(samples: list):
    questions = samples['question']
    ground_truth = samples['answer']
    
    input_ids = []
    query = []
    
    for question in questions:
        prompted_input = f'question: {question}\nanswer:'
        query.append(prompted_input)
        tokenized_input = tokenizer(prompted_input, 
                                    truncation=True)
        input_ids.append(torch.tensor(tokenized_input['input_ids'], dtype=torch.long))
        
    return {'input_ids': input_ids, 'query': query, 'ground_truth': ground_truth, 'questions': questions}



dataset = dataset.map(tokenize, 
                      batched=True, 
                      #num_proc=num_proc, 
                      load_from_cache_file=False, 
                      remove_columns=['question', 'answer'])
dataset.set_format('pt', 
                   columns=['input_ids', 'query', 'ground_truth'],
                   output_all_columns=True)
dataset

Map:   0%|          | 0/2081 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'query', 'ground_truth', 'questions'],
    num_rows: 2081
})

In [55]:
def collator(dataset):
    result = {}
    for key in dataset[0]:
        values = []
        for d in dataset:
            values.append(d[key])
        result[key] = values
    return result

# creates an instance of the PPOTrainer class
ppo_trainer = PPOTrainer(config, active_model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

eval/loss,▁▁▁▁▂▂▃▄▄▃▄▄▄▄▄▄▅▄▅▅▇▆▆▇▆▆▆▆▆▆▇▇▇▇▇▇████
eval/runtime,▆▁▃▂▂▂▂▃▂█▇▆▄▆▆▅▄▂▄▇▇▁▅▅▄▆▆▂▄▃▃▃▃▃▆▄▃▂▄▂
eval/samples_per_second,▃▇▆▇▇▆▇▆▇▁▂▃▄▃▃▄▅▇▅▂▁█▄▄▅▃▃▇▅▅▆▆▆▆▃▄▅▆▅▇
eval/steps_per_second,▃▇▆▇▇▆▇▆▇▁▂▃▄▃▃▄▅▇▅▂▁█▄▄▅▃▃▇▅▅▆▆▆▆▃▄▅▆▅▇
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/grad_norm,▁▁▁▁▁█▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,██▇▆▅▄▄▃▃▃▃▂▂▂▂▁▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [56]:
bert_pipe = pipeline('text-generation', 
                     model=args.output_dir,
                    max_length = args.MAX_LEN)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [57]:
import json
prompt = "Are there clubs and associations that students can join?"
output = bert_pipe(prompt, max_length=512, num_return_sequences=1)
output = output[0]['generated_text']
response_text = output.split('[RESPONSE]')[1]
formatted_text = "```\n" + response_text + "\n```"
print(formatted_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


```
Weberf, Syms, and the Sy Syms School of Business.
```


In [58]:
import json
# prompt = "Are there clubs and associations that students can join?"
prompt ="How active is the student community on campus?"
output = bert_pipe(prompt, max_length=512, num_return_sequences=1)
output = output[0]['generated_text']
response_text = output.split('[RESPONSE]')[1]
formatted_text = "```\n" + response_text + "\n```"
print(formatted_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


```
The student community on campus is vibrant and has been diverse for ages.
```


In [59]:
print(response_text)

The student community on campus is vibrant and has been diverse for ages.


In [60]:
import json

def generate_response(prompt: str, bert_pipe):
    # Generate response using the BERT-based pipeline
    output = bert_pipe(prompt, max_length=512, num_return_sequences=1, pad_token_id=50256)
    output = output[0]['generated_text']
    response_text = output.split('[RESPONSE]')[1].strip() if output else "No response found"
    return response_text

# Example usage:
prompt = "How active is the student community on campus?"
response = generate_response(prompt, bert_pipe)
print(response)


Yes, the student community on campus is vibrant and diverse, with over 70,000 people.


In [61]:
test_df_rlhf = pd.read_csv(args.test_data)
# df = df.iloc[101:]
print(test_df_rlhf.shape)
test_df_rlhf.head()

(2081, 2)


,question,answer
0,What is the student/faculty ratio at this university?,The student/faculty ratio at Yeshiva University is 7:1.
1,How often do students get to interact with professors outside of class?,"Students at Yeshiva University have ample opportunities to interact with professors outside of class through office hours, research projects, and mentoring programs."
2,How active is the student community on campus?,"The student community at Yeshiva University is highly active, with various clubs, organizations, and events that foster a strong sense of community on campus."
3,Can you tell me more about the extracurricular activities offered at this university?,"Yeshiva University offers a wide range of extracurricular activities, including cultural clubs, academic societies, volunteer opportunities, and sports teams."
4,What percentage of students get financial aid?,Approximately 85% of students at Yeshiva University receive some form of financial aid.


In [62]:
test_df_rlhf['question'][1002]

'Can you tell me about yourself?'

In [63]:
import json

def generate_response(prompt: str, bert_pipe):
    # Generate response using the BERT-based pipeline
    output = bert_pipe(prompt, max_length=512, num_return_sequences=1, pad_token_id=50256)
    output = output[0]['generated_text']
    response_text = output.split('[RESPONSE]')[1].strip() if output else "No response found"
    return response_text

# Example usage:
prompt = test_df_rlhf['question'][1002]
response = generate_response(prompt, bert_pipe)
print(response)


Im Katzbot, created by students from the Artificial Intelligence program at Katz School at Yeshiva under Prof. Youshan Zhangs supervision. My primary goal is to provide students with rapid and accurate responses to their inquiries about our school.


In [64]:
def generate_response(prompt: str, bert_pipe):
    output = bert_pipe(prompt, max_length=512, num_return_sequences=1, pad_token_id=50256)
    
    # Check if output is empty
    if output and len(output) > 0:
        output_text = output[0].get('generated_text', '')
        # Check if the delimiter is present in output_text
        if '[RESPONSE]' in output_text:
            response_text = output_text.split('[RESPONSE]')[1].strip()
        else:
            response_text = "No response found"
    else:
        response_text = "No response found"
    
    return response_text

text_file_rlhf = os.path.join(args.output_dir, args.save_text_file_rlhf)
answers_rlhf = []

with open(text_file_rlhf, 'w') as f:
    for i, row in tqdm(test_df_rlhf.iterrows(), total=len(test_df_rlhf)):
        question, answer = row
        answer_rlhf = generate_response(question, bert_pipe)
        answers_rlhf.append(answer_rlhf)
        f.write(f"Question {i+1}/{len(test_df_rlhf)} => {question}\n")
        f.write(f"Answer => {answer}\n")
        f.write(f"Predicted Answer => {answer_rlhf}\n")
        f.write('\n\n')

# Optionally, you can print a message indicating that the output has been saved
print(f"Output saved to {text_file_rlhf}")


100%|██████████| 2081/2081 [45:20<00:00,  1.31s/it]

Output saved to models_pretrained_4\predicted_answer_rlhf.txt


In [65]:
# Add the predicted answers as a new column in the dataframe
test_df_rlhf[args.predicted_result] = answers_rlhf

test_predict_rlhf = os.path.join(args.output_dir, args.predict_file)

# Save the updated dataframe to a new CSV file
test_df_rlhf.to_csv(test_predict_rlhf, index=False)

print(len(answers_rlhf))
# Load the QApairs.csv file into a pandas dataframe
preds_rlhf = pd.read_csv(test_predict_rlhf)

preds_rlhf.head()

2081


,question,answer,predicted_answer
0,What is the student/faculty ratio at this university?,The student/faculty ratio at Yeshiva University is 7:1.,The student was awarded a Master of Science degree by the University of Kansas SpeechLanguageHearing Association.
1,How often do students get to interact with professors outside of class?,"Students at Yeshiva University have ample opportunities to interact with professors outside of class through office hours, research projects, and mentoring programs.",Students get to interact with professors outside of class.
2,How active is the student community on campus?,"The student community at Yeshiva University is highly active, with various clubs, organizations, and events that foster a strong sense of community on campus.","Yes, the student community on campus is vibrant, with over 70,000 Yeshiva University alumni and over 70,000 alumni."
3,Can you tell me more about the extracurricular activities offered at this university?,"Yeshiva University offers a wide range of extracurricular activities, including cultural clubs, academic societies, volunteer opportunities, and sports teams.",You can tell me more about the opportunities offered at this university by contacting the Student Council or the specific staff members listed above for their respective roles
4,What percentage of students get financial aid?,Approximately 85% of students at Yeshiva University receive some form of financial aid.,Over 80 of students receive financial assistance.


In [66]:
# calculating rogue score for answer 1
metrics_df = calculate_rouge_scores(preds_rlhf, args.original_text, args.predicted_result)
metrics_df

,Metric,Precision,Recall,F1-Score
0,rouge-1,0.529173,0.451392,0.472653
1,rouge-2,0.388525,0.357131,0.364392
2,rouge-l,0.506200,0.436863,0.455598


In [67]:
bertscore_gpt_rlhf = bertscore.compute(predictions=answers_rlhf, 
                                           references=test_df_rlhf['answer'].to_list(), 
                                           lang='en')['f1']

test_df_rlhf['bert_score_rlhf'] = bertscore_gpt_rlhf
test_df_rlhf.head()

,question,answer,predicted_answer,bert_score_rlhf
0,What is the student/faculty ratio at this university?,The student/faculty ratio at Yeshiva University is 7:1.,The student was awarded a Master of Science degree by the University of Kansas SpeechLanguageHearing Association.,0.839125
1,How often do students get to interact with professors outside of class?,"Students at Yeshiva University have ample opportunities to interact with professors outside of class through office hours, research projects, and mentoring programs.",Students get to interact with professors outside of class.,0.913571
2,How active is the student community on campus?,"The student community at Yeshiva University is highly active, with various clubs, organizations, and events that foster a strong sense of community on campus.","Yes, the student community on campus is vibrant, with over 70,000 Yeshiva University alumni and over 70,000 alumni.",0.906102
3,Can you tell me more about the extracurricular activities offered at this university?,"Yeshiva University offers a wide range of extracurricular activities, including cultural clubs, academic societies, volunteer opportunities, and sports teams.",You can tell me more about the opportunities offered at this university by contacting the Student Council or the specific staff members listed above for their respective roles,0.834625
4,What percentage of students get financial aid?,Approximately 85% of students at Yeshiva University receive some form of financial aid.,Over 80 of students receive financial assistance.,0.924429


In [68]:
np.mean(test_df_rlhf['bert_score_rlhf'])

0.9069235727570482